# Resize images

In [3]:
from PIL import Image
import os

def resize_images(input_folder, output_folder, width, height):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, filename in enumerate(os.listdir(input_folder)):
        try:
            with Image.open(os.path.join(input_folder, filename)) as img:
                resized_img = img.resize((width, height))
                resized_img.save(os.path.join(output_folder, str(i) + '.jpg'))
        except Exception as e:
            print(f"Failed to process {filename}: {e}")

# Set your desired width and height
desired_width = 2000
desired_height = 3000

# Replace 'input_folder' and 'output_folder' with your folder paths
input_folder_path = 'img/trips/kiawah_2023/'
output_folder_path = 'img/trips/kiawah_2023_2/'

resize_images(input_folder_path, output_folder_path, desired_width, desired_height)